In [104]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = [16,9]
%matplotlib inline

In [105]:
#Parsing date column of both datasets to be read by python

pdata = pd.read_csv('pollution-1.csv',parse_dates=['ReadingDateTime'])
wdata = pd.read_csv('weather-1.csv',parse_dates=['DATE'])

In [106]:
#Extracting numerical values of dew point, temperature, wind speed (converting from mps to kmph) and direction

wdata['DEW'] = wdata['DEW'].str[:-2].astype(np.float64)/10
wdata['TMP'] = wdata['TMP'].str[:-2].astype(np.float64)/10
wdata['DIR'] = wdata['WND'].str[:3].astype(np.float64)
wdata['SPD'] = (wdata['WND'].str[8:-2].astype(np.float64)/10)*3.6

#Calculating relative humidity from dew point and temperature

wdata['HUM'] = 100*(np.exp((17.625 * wdata['DEW'])/(243.04 + wdata['DEW']))/np.exp((17.625 * wdata['TMP'])/(243.04 + wdata['TMP'])))

#Replacing missing wind direction observations with nulls

wdata.DIR.replace(999,np.nan,inplace=True)

In [107]:
#Downsampling the weather dataset

wdata.index=wdata['DATE']
wdata_resampled = wdata.resample('1H').mean()[:17520]
wdata = wdata_resampled.reset_index()

In [108]:
#Joining the weather and pollution datasets

data = pd.concat([wdata['DATE'], wdata['DEW'],wdata['TMP'],wdata['DIR'],wdata['SPD'],wdata['HUM'],pdata.Value[pdata.Species=='NO2'].reset_index(drop=True).rename('NO2'),pdata.Value[(pdata.Species=='O3')].reset_index(drop=True).rename('O3'),pdata.Value[(pdata.Species=='PM2.5')].reset_index(drop=True).rename('PM25')], axis=1)

In [109]:
#Number of outilers

print('DEW:',((data['DEW'] <= -99) | (data['DEW'] >= 37)).sum())

print('TMP:',((data['TMP'] <= -94) | (data['TMP'] >= 62)).sum())

print('DIR:',((data['DIR'] < 1) | (data['DIR'] > 360)).sum())

print('SPD:',((data['SPD'] < 0) | (data['SPD'] > 324)).sum())

print('HUM:',((data['HUM'] < 0) | (data['HUM'] > 100)).sum())

print('NO2:',((data['NO2'] < 0) | (data['NO2'] > 601)).sum())

print('O3:',((data['O3'] < 0) | (data['O3'] > 241)).sum())

print('PM25:',((data['PM25'] < 0) | (data['PM25'] > 71)).sum())

DEW: 9
TMP: 9
DIR: 0
SPD: 10
HUM: 0
NO2: 0
O3: 9
PM25: 128


In [110]:
#Old method was not ideal
# data['DEW'].loc[(data['DEW'] <= -99) | (data['DEW'] >= 37)] = np.nan
# data['TMP'].loc[(data['TMP'] <= -94) | (data['TMP'] >= 62)] = np.nan
# data['SPD'].loc[(data['SPD'] < 0) | (data['SPD'] > 324)] = np.nan
# data['O3'].loc[(data['O3'] < 0) | (data['O3'] > 241)] = np.nan
# data['PM25'].loc[(data['PM25'] < 0) | (data['PM25'] > 71)] = np.nan

In [111]:
#Replacing outliers with nulls

dew_out = data.DEW[(data['DEW'] <= -99) | (data['DEW'] >= 37)]
tmp_out = data.TMP[(data['TMP'] <= -94) | (data['TMP'] >= 62)]
spd_out = data.SPD[(data['SPD'] < 0) | (data['SPD'] > 324)]
o3_out = data.O3[(data['O3'] < 0) | (data['O3'] > 241)]
pm25_out = data.PM25[(data['PM25'] < 0) | (data['PM25'] > 71)]
data.DEW.replace(dew_out,np.nan,inplace=True)
data.TMP.replace(tmp_out,np.nan,inplace=True)
data.SPD.replace(spd_out,np.nan,inplace=True)
data.O3.replace(o3_out,np.nan,inplace=True)
data.PM25.replace(pm25_out,np.nan,inplace=True)

In [112]:
#Adding mask columns to filter null values to make dropping/filling easier

# data['DEW_MASK'] = data['DEW'].isna()
# data['TMP_MASK'] = data['TMP'].isna()
# data['DIR_MASK'] = data['DIR'].isna()
# data['SPD_MASK'] = data['SPD'].isna()
# data['HUM_MASK'] = data['HUM'].isna()
# data['NO2_MASK'] = data['NO2'].isna()
# data['O3_MASK'] = data['O3'].isna()
# data['PM25_MASK'] = data['PM25'].isna()

In [113]:
# Add code here to fill null values

In [114]:
#Adding date features

data['MONTH'] = pd.DatetimeIndex(data['DATE']).month
data['HOUR'] = pd.DatetimeIndex(data['DATE']).hour
data['DAY'] = data['DATE'].dt.weekday_name
data['DAY_CAT'] = data.DAY.astype("category").cat.codes

In [115]:
# Adding time shifted features
## This is done before removing nulls so that the dataset is continuous
### To be cleaned up into a function

data['DEW_1'] = data.DEW.shift(periods=1)
data['TMP_1'] = data.TMP.shift(periods=1)
data['DIR_1'] = data.DIR.shift(periods=1)
data['SPD_1'] = data.SPD.shift(periods=1)
data['HUM_1'] = data.HUM.shift(periods=1)
data['NO2_1'] = data.NO2.shift(periods=1)
data['O3_1'] = data.NO2.shift(periods=1)
data['PM25_1'] = data.NO2.shift(periods=1)

data['DEW_2'] = data.DEW.shift(periods=2)
data['TMP_2'] = data.TMP.shift(periods=2)
data['DIR_2'] = data.DIR.shift(periods=2)
data['HUM_4'] = data.HUM.shift(periods=2)
data['SPD_2'] = data.SPD.shift(periods=2)
data['NO2_2'] = data.NO2.shift(periods=2)
data['O3_2'] = data.NO2.shift(periods=2)
data['PM25_2'] = data.NO2.shift(periods=2)

data['DEW_3'] = data.DEW.shift(periods=3)
data['TMP_3'] = data.TMP.shift(periods=3)
data['DIR_3'] = data.DIR.shift(periods=3)
data['HUM_4'] = data.HUM.shift(periods=3)
data['SPD_3'] = data.SPD.shift(periods=3)
data['NO2_3'] = data.NO2.shift(periods=3)
data['O3_3'] = data.NO2.shift(periods=3)
data['PM25_3'] = data.NO2.shift(periods=3)

data['DEW_4'] = data.DEW.shift(periods=4)
data['TMP_4'] = data.TMP.shift(periods=4)
data['DIR_4'] = data.DIR.shift(periods=4)
data['HUM_4'] = data.HUM.shift(periods=4)
data['SPD_4'] = data.SPD.shift(periods=4)
data['NO2_4'] = data.NO2.shift(periods=4)
data['O3_4'] = data.NO2.shift(periods=4)
data['PM25_4'] = data.NO2.shift(periods=4)

data['DEW_5'] = data.DEW.shift(periods=5)
data['TMP_5'] = data.TMP.shift(periods=5)
data['DIR_5'] = data.DIR.shift(periods=5)
data['HUM_5'] = data.HUM.shift(periods=5)
data['SPD_5'] = data.SPD.shift(periods=5)
data['NO2_5'] = data.NO2.shift(periods=5)
data['O3_5'] = data.NO2.shift(periods=5)
data['PM25_5'] = data.NO2.shift(periods=5)

In [ ]:
#Checking number of null values

data.isna().sum()

In [ ]:
#Percentage of null values in final dataset

data.isna().mean().round(4) * 100

In [92]:
# Show rows where any cell has a NaN
data[data.isnull().any(axis=1)].shape

(5656, 51)

In [116]:
#dropping all null values

data_dropped = data.dropna(axis=0).reset_index(drop=True)

In [118]:
#dropping unwanted columns to create final dataset for model
data_final_dropped = data_dropped.drop(['DATE','DAY'], axis=1)

In [120]:
data_final_dropped.to_csv('final-dropped.csv',index=False)